In [1]:
# ============= 安装依赖（Colab 第一格运行） =============
!pip -q install geopandas shapely pyproj fiona rtree python-igraph tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 23.7 MB/s eta 0:00:00


In [4]:
# ============= 导入包 =============
import geopandas as gpd
import numpy as np
import igraph as ig
from tqdm.auto import tqdm
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/CASA0004_Cycling/data'

Mounted at /content/drive
BoroughShp  GreatLondonShp  s1	s2_Env	s3


In [3]:
# ============= 参数区（按需改） =============
# 输入：伦敦范围已裁剪好的路网（线要素）
IN_GPKG = "/content/drive/MyDrive/CASA0004_Cycling/data/s3/codetest.gpkg"  # 也可以是 .geojson
IN_LAYER = None  # 如果是 GPKG，多图层时填具体图层名；单图层或 geojson 填 None

# 输出
OUT_GEOJSON = "/content/codetest_with_centrality.gpkg"

In [5]:
ziduan = gpd.read_file(IN_GPKG)
print(ziduan.columns)

Index(['id', 'name', 'way_type', 'index', 'index_10', 'stress_level', 'offset',
       'side', 'proc_width', 'proc_surface', 'proc_smoothness', 'proc_oneway',
       'proc_sidepath', 'proc_highway', 'proc_maxspeed',
       'proc_traffic_mode_left', 'proc_traffic_mode_right',
       'proc_separation_left', 'proc_separation_right', 'proc_buffer_left',
       'proc_buffer_right', 'proc_mandatory', 'proc_traffic_sign', 'fac_width',
       'fac_surface', 'fac_highway', 'fac_maxspeed', 'base_index', 'fac_1',
       'fac_2', 'fac_3', 'fac_4', 'data_bonus', 'data_malus',
       'data_incompleteness', 'data_missing', 'data_missing_width',
       'data_missing_surface', 'data_missing_smoothness',
       'data_missing_maxspeed', 'data_missing_parking', 'data_missing_lit',
       'filter_usable', 'filter_way_type', 'geometry'],
      dtype='object')


In [6]:
# 长度权重字段名（如果没有，就用 geometry.length）
LEN_FIELD = "metres"    # 若没有该字段，会自动改用 geometry.length
TOL = 1.0               # 端点量化容差（米），用于“吸附”断点
SEED = 42               # 随机种子

# betweenness 计算模式
BET_MODE = "approx"     # "exact" 或 "approx"
K_SAMPLES = 1200        # 近似模式下采样源点数量（建议：500~3000 之间按机器调）

# closeness 选项
CLOSENESS_HARMONIC = True   # 非连通图建议用 harmonic 口径

In [7]:
# ============= 读取与预处理 =============
if IN_LAYER:
    roads = gpd.read_file(IN_GPKG, layer=IN_LAYER)
else:
    roads = gpd.read_file(IN_GPKG)

# 修正/设置 CRS（OpenMapping 通常是 EPSG:27700）
if roads.crs is None:
    roads = roads.set_crs(27700)
elif str(roads.crs).endswith("4326"):
    # 若误读成经纬度，通常需要改回 27700；你也可以根据 bounds 判断再 set_crs
    roads = roads.set_crs(27700, allow_override=True)

# 单部件化，清理空几何
roads = roads.explode(index_parts=False, ignore_index=True)
roads = roads[~roads.geometry.is_empty & roads.geometry.notna()].copy()

# 生成长度字段
if LEN_FIELD in roads.columns:
    roads["length_m"] = roads[LEN_FIELD].astype(float)
else:
    roads["length_m"] = roads.geometry.length

In [8]:
# ============= 建图：端点 -> 节点；路段 -> 边 =============
def qpt(xy, tol=TOL):
    return (round(xy[0]/tol)*tol, round(xy[1]/tol)*tol)

# 映射端点坐标到节点索引
node_index = {}
nodes = []
edges = []
weights = []

for geom, w in tqdm(zip(roads.geometry, roads["length_m"]), total=len(roads), desc="Build graph"):
    coords = geom.coords
    u_xy = qpt(coords[0]); v_xy = qpt(coords[-1])
    for xy in (u_xy, v_xy):
        if xy not in node_index:
            node_index[xy] = len(nodes)
            nodes.append(xy)
    u = node_index[u_xy]; v = node_index[v_xy]
    edges.append((u, v))
    weights.append(float(w))

g = ig.Graph()
g.add_vertices(len(nodes))
g.add_edges(edges)
g.es["length"] = weights  # 加权最短路会使用该属性

# 只保留最大连通子图（可选）
components = g.clusters()
gi = components.giant()
# 建立从 giant 子图回到原始边的映射
gi_edge_mask = np.zeros(len(edges), dtype=bool)
# 找出保留边（子图的边对应原图一子集；我们重建一次映射）
# 简单做法：用端点坐标反查
gi_nodes_idx = set(gi.vs["_graph_id"]) if "_graph_id" in gi.vs.attributes() else set()
# 若没有 _graph_id，则直接对 giant 的边按端点坐标比对
gi_coords = [nodes[v.index] for v in gi.vs]
coord_to_new_idx = {xy: i for i, xy in enumerate(gi_coords)}

Build graph:   0%|          | 0/7280 [00:00<?, ?it/s]

/tmp/ipython-input-3751521312.py:28: DeprecationWarning: Graph.clusters() is deprecated; use Graph.connected_components() instead
  components = g.clusters()


In [10]:
# ============= Closeness（Harmonic，多尺度）并映射到边（两端点均值） =============
import numpy as np
from tqdm.auto import tqdm

def harmonic_closeness(graph, weight_attr="length", radii=None):
    """
    计算加权 Harmonic Closeness（节点级），可选多尺度半径版本。
    radii: None 或 [2000, 10000, 100000]（单位与坐标一致，这里是米）
    返回 dict：键为 'global' 或 'R=2000' 等，值为长度=vcount 的 ndarray
    """
    # 全局最短路矩阵（v x v）
    dist = np.array(graph.shortest_paths(weights=weight_attr), dtype=float)
    np.fill_diagonal(dist, np.nan)  # 自身距离不计

    out = {}
    # Global（不设半径）
    inv = 1.0 / dist
    inv[~np.isfinite(inv)] = np.nan
    out['global'] = np.nansum(inv, axis=1)

    # 多半径
    if radii:
        for R in radii:
            mask = dist <= float(R)
            invR = np.where(mask, 1.0 / dist, np.nan)
            out[f'R={R}'] = np.nansum(invR, axis=1)

    return out

def edge_from_node_avg(graph, node_vals):
    """将节点值映射到边（两端点均值）"""
    vals = []
    for e in graph.es:
        s, t = e.tuple
        vals.append((node_vals[s] + node_vals[t]) / 2.0)
    return np.array(vals)

# ======== 调用 ========
with tqdm(total=1, desc="Harmonic closeness") as pbar:
    hc = harmonic_closeness(gi, weight_attr="length", radii=[2000, 10000, 100000])
    pbar.update(1)

# 将多尺度 closeness 写入边属性
gi.es["closeness_hc_global"] = edge_from_node_avg(gi, hc['global']).tolist()
gi.es["closeness_hc_2km"]    = edge_from_node_avg(gi, hc['R=2000']).tolist()
gi.es["closeness_hc_10km"]   = edge_from_node_avg(gi, hc['R=10000']).tolist()
gi.es["closeness_hc_100km"]  = edge_from_node_avg(gi, hc['R=100000']).tolist()


Harmonic closeness:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipython-input-1560021919.py:12: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  dist = np.array(graph.shortest_paths(weights=weight_attr), dtype=float)


In [11]:
# ============= Betweenness（边） =============
random.seed(SEED)
np.random.seed(SEED)

if BET_MODE == "exact":
    # 可能较慢，但 igraph 比 networkx 快
    with tqdm(total=1, desc="Edge betweenness (exact)") as pbar:
        eb = gi.edge_betweenness(weights="length")
        gi.es["betweenness_edge"] = eb
        pbar.update(1)
else:
    # 近似：按源点采样 K_SAMPLES，累计最短路经过次数（权重=length）
    # 思路：对每个源 s，算到所有 t 的最短路，拿到“边路径”(epath)，遇到的边计数+1
    # 注意：这是“最短路条数计数”的近似，和 Brandes 规范化的 betweenness 口径不同，但与 Choice 思路接近
    m = gi.ecount()
    counts = np.zeros(m, dtype=np.float64)

    all_nodes = list(range(gi.vcount()))
    if K_SAMPLES > gi.vcount():
        K_SAMPLES = gi.vcount()
    sources = random.sample(all_nodes, K_SAMPLES)

    for s in tqdm(sources, desc=f"Edge betweenness approx (K={K_SAMPLES})"):
        # 到所有节点的最短路，返回边索引路径列表；不可达返回空
        epaths = gi.get_shortest_paths(s, to=all_nodes, weights="length", output="epath")
        for epath in epaths:
            if not epath:  # 自身或不可达
                continue
            counts[epath] += 1.0

    # 归一化（可按需要换不同规范化口径）
    counts /= K_SAMPLES
    gi.es["betweenness_edge"] = counts.tolist()

Edge betweenness approx (K=1200):   0%|          | 0/1200 [00:00<?, ?it/s]

In [13]:
import numpy as np
from tqdm.auto import tqdm

# 选一个要回写到 roads 的 closeness 字段（在 gi.es 里可用的字段名）
# 可选："closeness_hc_global", "closeness_hc_2km", "closeness_hc_10km", "closeness_hc_100km"
CLOSE_ATTR = "closeness_hc_10km"   # ← 按需改

# 看看边属性里有哪些可用字段
print("Edge attributes available:", gi.es.attributes())
if CLOSE_ATTR not in gi.es.attributes():
    raise ValueError(f"边属性里找不到 {CLOSE_ATTR}，请检查前面 harmonic closeness 的生成步骤。")

# === 将 giant 子图的边指标映射回原始 GeoDataFrame ===
# 1) 建立：子图节点坐标列表 & 坐标 → 子图节点索引 映射
gi_xy = [nodes[v.index] for v in gi.vs]  # 子图节点对应原坐标
xy_to_gi = {xy: i for i, xy in enumerate(gi_xy)}

# 2) 建立：子图的 (端点坐标对) → 子图边索引 映射（无向图双向登记）
edge_dict = {}
for e in gi.es:
    s, t = e.tuple
    xy_s = gi_xy[s]; xy_t = gi_xy[t]
    edge_dict[(xy_s, xy_t)] = e.index
    edge_dict[(xy_t, xy_s)] = e.index

bet_vals = []
close_vals = []

for geom in tqdm(roads.geometry, desc="Map metrics back"):
    if geom is None or geom.is_empty:
        bet_vals.append(np.nan); close_vals.append(np.nan); continue

    # 端点坐标量化要与建图阶段一致
    u_xy = (round(geom.coords[0][0]/TOL)*TOL, round(geom.coords[0][1]/TOL)*TOL)
    v_xy = (round(geom.coords[-1][0]/TOL)*TOL, round(geom.coords[-1][1]/TOL)*TOL)

    su = xy_to_gi.get(u_xy); tv = xy_to_gi.get(v_xy)
    if su is None or tv is None:
        bet_vals.append(np.nan); close_vals.append(np.nan); continue

    eidx = edge_dict.get((u_xy, v_xy))
    if eidx is None:
        bet_vals.append(np.nan); close_vals.append(np.nan); continue

    # 取 betweenness（可能是 exact 或 approx 版本）
    bet = gi.es[eidx]["betweenness_edge"] if "betweenness_edge" in gi.es.attributes() else np.nan
    # 取你选择的 harmonic closeness 尺度
    clo = gi.es[eidx][CLOSE_ATTR]

    bet_vals.append(bet)
    close_vals.append(clo)

# 回写到 GeoDataFrame
roads["edge_betweenness"] = bet_vals
roads[f"edge_{CLOSE_ATTR}"] = close_vals


Edge attributes available: ['length', 'closeness_hc_global', 'closeness_hc_2km', 'closeness_hc_10km', 'closeness_hc_100km', 'betweenness_edge']


Map metrics back:   0%|          | 0/7280 [00:00<?, ?it/s]

In [14]:
# ============= 导出 =============
roads.to_file(OUT_GEOJSON, driver="gpkg")
print("Saved:", OUT_GEOJSON)
print("Rows:", len(roads))

Saved: /content/codetest_with_centrality.gpkg
Rows: 7280
